In [80]:
import pandas as pd 
import numpy as np 
import xgboost
import os
import time
import warnings
import sklearn
import seaborn as sns 
from sklearn.decomposition import PCA
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt 
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')
file=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/matrix_0.csv'
weatherfile=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/weather_data_2014-1-1~2016~9-30.xlsx'
holidayfile=r'c:/users/lance/desktop/GitHub_project/tianchi_power_data/holidays.xlsx'

In [81]:
holidaydata=pd.read_excel(holidayfile,header=None)
data=pd.read_csv(file)
weatherdata=pd.read_excel(weatherfile)
k=['timestamp']
k+=[str(x) for x in range(1,1455)]
k.append('sum')
data.columns=k
data['timestamp']=pd.DataFrame(pd.to_datetime(data['timestamp']))

In [82]:
data['dayofweek']=data['timestamp'].apply(lambda x:x.dayofweek)
data['dayofyear']=data['timestamp'].apply(lambda x:x.dayofyear)
data['is_weekend']=data['timestamp'].apply(lambda x:1 if x.dayofweek in range(5,7) else 0)

In [83]:
data['num_working']=1454
for i in range(0,609):
    data.loc[i,'num_working']=1454-Counter(data.loc[i,][1:1455])[1]

In [84]:
holidaylist=list(holidaydata[0])

data['is_holiday']=data.timestamp.apply(lambda x:1 if x in holidaylist else 0)

In [85]:
data=pd.merge(left=data,right=weatherdata,left_on='timestamp',right_on='date',how='left')

pd.get_dummies(data['weather'])

,中到大雨~中雨,中到大雨~小雨,中到大雨~暴雨,中到大雨~阴,中到大雨转小雨,中雨,中雨~中到大雨,中雨~多云,中雨~大雨,中雨~小雨,...,雨夹雪转阴,雷阵雨,雷阵雨~中雨,雷阵雨~多云,雷阵雨~大到暴雨,雷阵雨~小到中雨,雷阵雨~阴,雷阵雨转多云,雷雨,雷雨~大雨
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
df=data[['timestamp','sum','dayofweek','dayofyear','is_weekend','num_working','is_holiday','max','min','max-min']]

In [87]:
df['sum_shift30']=df['sum'].shift(30)

In [88]:
df=df.loc[30:,]

In [89]:
df

,timestamp,sum,dayofweek,dayofyear,is_weekend,num_working,is_holiday,max,min,max-min,sum_shift30
30,2015-01-31,3895604,5,31,1,1125,0,5.0,-1.0,6.0,2900575.0
31,2015-02-01,3678729,6,32,1,1126,0,7.0,1.0,6.0,3158211.0
32,2015-02-02,3983918,0,33,0,1130,0,7.0,2.0,5.0,3596487.0
33,2015-02-03,3962811,1,34,0,1128,0,7.0,2.0,5.0,3939672.0
34,2015-02-04,3904604,2,35,0,1127,0,7.0,-2.0,9.0,4101790.0
35,2015-02-05,3812319,3,36,0,1132,0,9.0,-2.0,11.0,4149164.0
36,2015-02-06,3808081,4,37,0,1127,0,12.0,-1.0,13.0,4161928.0
37,2015-02-07,3503405,5,38,1,1130,0,6.0,0.0,6.0,4182622.0
38,2015-02-08,3476838,6,39,1,1125,0,6.0,-4.0,10.0,4153509.0
39,2015-02-09,3562045,0,40,0,1122,0,15.0,0.0,15.0,3913704.0


In [90]:
df=df.loc[:609]

In [91]:
train=df[df.timestamp<r'2016/07/01']
test=df[(df.timestamp>=r'2016/07/01')&(df.timestamp<r'2016/08/01')]

In [92]:
xtrain=train[['sum','dayofweek','dayofyear','is_weekend','num_working','is_holiday']]
ytrain=train[['sum_shift30']]

In [93]:
xtest=test[['sum','dayofweek','dayofyear','is_weekend','num_working','is_holiday']]
ytest=test[['sum_shift30']]

In [94]:
from sklearn.ensemble import RandomForestRegressor
import xgboost

In [95]:
#rfr=RandomForestRegressor(n_estimators=100,verbose=1,random_state=1,max_features=3)
rfr=xgboost.sklearn.XGBRegressor(n_estimators=1000,learning_rate=0.11,gamma=1,base_score=0.7,max_depth=2)
rfr.fit(xtrain,ytrain)

result=rfr.predict(xtest)

res=pd.concat([pd.DataFrame(df[df.timestamp>=r'2016/08/01']['sum']).reset_index(),pd.DataFrame(result)],axis=1)

del res['index']

res.columns=['true','pre']

res['bias']=res['true']-res['pre']
res['mean2']=res['bias']**2
sum(res['mean2'])/len(res)

138478110081.19958